# Database connection

### Functions to run queries

In [3]:
import mysql.connector
import pandas as pd

def connect():
    con = mysql.connector.connect(
        host="35.193.209.4",
        user="0213663",
        password="0213663",
        database="up_0213663"
    )
    cur = con.cursor()
    return con, cur

def close(con, cur):
    cur.close()
    con.close()
    
def run(query):
    con, cur = connect()
    cur.execute(query)
    close(con, cur)
    
def retrieve(query):
    con, cur = connect()
    cur.execute(query)
    rows = cur.fetchall()
    columns = [d[0] for d in cur.description]
    close(con, cur)
    return pd.DataFrame(rows, columns=columns)

## Tables

The design of our database was structured thinking about how the interactions between them would be easier to progrram in the Django project. The result is shown as follows:

![DB](DB.png)

### Query to create tables

```sql
CREATE TABLE `students` (
  `id` CHAR(10) PRIMARY KEY NOT NULL,
  `name` VARCHAR(128) NOT NULL,
  `last_names` VARCHAR(128) NOT NULL,
  `alias` VARCHAR(128)
);

CREATE TABLE `classes` (
  `class_number` INT AUTO_INCREMENT PRIMARY KEY NOT NULL,
  `topic` VARCHAR(128) NOT NULL,
  `created_at` TIMESTAMP NOT NULL DEFAULT CURRENT_TIME
);

CREATE TABLE `questions` (
  `class_number` INT PRIMARY KEY NOT NULL,
  `question` VARCHAR(256),
);

CREATE TABLE `answers` (
  `class_number` INT PRIMARY KEY NOT NULL,
  `student_id` CHAR(10) NOT NULL,
  `given_answer` VARCHAR(256) NOT NULL
);

CREATE TABLE `attendance` (
  `class` INT PRIMARY KEY NOT NULL,
  `student_id` VARCHAR(256) NOT NULL,
  `attendance` bool NOT NULL DEFAULT 0
);

CREATE TABLE `quiz` (
  `student_id` CHAR(10) PRIMARY KEY,
  `question1` VARCHAR(256),
  `question2` VARCHAR(256),
  `question3` VARCHAR(256),
  `question4` VARCHAR(256),
  `question5` VARCHAR(256),
  `question6` VARCHAR(256),
  `answer1` VARCHAR(256),
  `answer2` VARCHAR(256),
  `answer3` VARCHAR(256),
  `answer4` VARCHAR(256),
  `answer5` VARCHAR(256),
  `answer6` VARCHAR(256)
);

CREATE TABLE `choices` (
  `question` VARCHAR(256) PRIMARY KEY,
  `choice` VARCHAR(256),
  `correct` bool NOT NULL DEFAULT 0
);
```

### Query to add relationships

```sql
ALTER TABLE `questions` ADD FOREIGN KEY (`class_number`) REFERENCES `classes` (`class_number`);
ALTER TABLE `questions` ADD FOREIGN KEY (`class_number`) REFERENCES `answers` (`class_number`);
ALTER TABLE `students` ADD FOREIGN KEY (`id`) REFERENCES `answers` (`student_id`);
ALTER TABLE `questions` ADD FOREIGN KEY (`question`) REFERENCES `answers` (`question`);
ALTER TABLE `classes` ADD FOREIGN KEY (`class_number`) REFERENCES `attendance` (`class_number`);
ALTER TABLE `students` ADD FOREIGN KEY (`id`) REFERENCES `attendance` (`student_id`);
ALTER TABLE `quiz` ADD FOREIGN KEY (`student_id`) REFERENCES `students` (`id`);
ALTER TABLE `choices` ADD FOREIGN KEY (`question`) REFERENCES `quiz` (`question1`);
ALTER TABLE `choices` ADD FOREIGN KEY (`question`) REFERENCES `quiz` (`question2`);
ALTER TABLE `choices` ADD FOREIGN KEY (`question`) REFERENCES `quiz` (`question3`);
ALTER TABLE `choices` ADD FOREIGN KEY (`question`) REFERENCES `quiz` (`question4`);
ALTER TABLE `choices` ADD FOREIGN KEY (`question`) REFERENCES `quiz` (`question5`);
ALTER TABLE `choices` ADD FOREIGN KEY (`question`) REFERENCES `quiz` (`question6`);
ALTER TABLE `choices` ADD FOREIGN KEY (`correct`) REFERENCES `quiz` (`answer1`);
ALTER TABLE `choices` ADD FOREIGN KEY (`correct`) REFERENCES `quiz` (`answer2`);
ALTER TABLE `choices` ADD FOREIGN KEY (`correct`) REFERENCES `quiz` (`answer3`);
ALTER TABLE `choices` ADD FOREIGN KEY (`correct`) REFERENCES `quiz` (`answer4`);
ALTER TABLE `choices` ADD FOREIGN KEY (`correct`) REFERENCES `quiz` (`answer5`);
ALTER TABLE `choices` ADD FOREIGN KEY (`correct`) REFERENCES `quiz` (`answer6`);
```

### Inserts per table (**examples**)

**students**
```sql
INSERT INTO students (id, name, last_names, alias) VALUES ('0224604', 'Paulina', 'Barba Mendoza', 'Pau'), ('0213663', 'Eduardo', 'Solano Jaime', 'Eduardo');
```
**classes**
```sql
INSERT INTO classes (class_number, topic) VALUES (1,'Django'), (2, 'Cloud Computing')
```
**questions**
```sql
INSERT INTO questions (class_number, question) VALUES (1, 'Favorite food');
```
**answers**
```sql
INSERT INTO answers (class_number, id, given_answer) VALUES (1, '0224604', 'Sushi'), (1, '0213663', 'Tacos');
```
**attendance**
```sql
INSERT INTO attendance (class_number, id, attendance) VALUES (1, '0224604', 1), (1, '0213663');
```
**quiz**
```sql
INSERT INTO quiz (student_id, question1, question2, question3, question4, question5, question6, answer1, answer2, answer3, answer4, answer5, answer6) 
VALUES ('0224604', 
        'Which SQL statement is used to retrieve data from a database?', 
        'What is the correct SQL statement to create a database named test_db?',
        'Which of the following is a valid data type in MySQL?',
        'What is the main purpose of an index in a database?',
        'Which SQL keyword is used to combine rows from two or more tables, based on a related column between them?',
        'Which SQL function is used to calculate the number of rows in a table?',
        1, 1, 0, 1, 0, 1), 
        ('0213663', 
        'Which SQL statement is used to retrieve data from a database?', 
        'What is the correct SQL statement to create a database named test_db?',
        'Which of the following is a valid data type in MySQL?',
        'What is the main purpose of an index in a database?',
        'Which SQL keyword is used to combine rows from two or more tables, based on a related column between them?',
        'Which SQL function is used to calculate the number of rows in a table?',
        1, 0, 0, 1, 0, 0) ;
```
**choices**
```sql
INSERT INTO attendance (question, choice, correct) 
VALUES ('Which SQL statement is used to retrieve data from a database?', 'GET', 0),
        ('Which SQL statement is used to retrieve data from a database?', 'RETRIEVE', 0),
        ('Which SQL statement is used to retrieve data from a database?', 'SELECT', 1),
        ('Which SQL statement is used to retrieve data from a database?', 'FETCH', 0);
```

### Run the queries

Fortunately, Django and the use of Models and Forms in python create the databases automatically in our server. We still have to translate the previous SQL statements in order to create the models and expect the same result in the DB, although the name of the tables will contain a suffix `myapp_` because of the name of the Django project.

In [19]:
retrieve('SHOW TABLES')[-7:]

,Tables_in_up_0213663
10,myapp_answer
11,myapp_attendance
12,myapp_choices
13,myapp_class
14,myapp_questions
15,myapp_quiz
16,myapp_student
